In [ ]:
# ======================================================================
# SELF-RAG + AGENTIC SELF-RAG — FULL WORKING SINGLE CELL (UNGATED MODEL)
# ======================================================================

!pip install transformers sentence-transformers faiss-cpu accelerate > /dev/null

import torch, faiss, numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer

# ----------------------------------------------------------------------
# 1. Load Free, Ungated LLM (Mistral-7B-Instruct)
# ----------------------------------------------------------------------
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.float16
)

# ----------------------------------------------------------------------
# 2. Build Retrieval System
# ----------------------------------------------------------------------
documents = [
    "Self-RAG adds adaptive retrieval and self-critique.",
    "It reduces hallucinations using reflection tokens.",
    "Agents perform planning, tool use, and multi-step execution.",
    "Self-RAG can verify answers using retrieved evidence.",
    "Self-RAG improves factual QA and reduces unsupported claims.",
    "Agents should check correctness before taking action."
]

embedder = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
doc_embeds = embedder.encode(documents, convert_to_numpy=True)
index = faiss.IndexFlatL2(doc_embeds.shape[1])
index.add(doc_embeds)

def retrieve(query, k=2):
    q = embedder.encode([query])
    D, I = index.search(q, k)
    return [documents[i] for i in I[0]]

# ----------------------------------------------------------------------
# 3. Helper function for the LLM
# ----------------------------------------------------------------------
def ask_llm(prompt, max_new_tokens=200):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    out = model.generate(**inputs, max_new_tokens=max_new_tokens)
    return tokenizer.decode(out[0], skip_special_tokens=True)

# ======================================================================
# 🔹 4. SELF-RAG (Baseline)
# ======================================================================
def self_rag(question):
    retrieve_prompt = f"""
Only answer 'yes' or 'no'.

Question: {question}
Do you need external retrieval?
"""
    decision = ask_llm(retrieve_prompt).strip().lower()

    print(" Self-RAG | Retrieval needed?", decision)

    evidence = ""
    if "yes" in decision:
        evidence = "\n".join(retrieve(question))

    final_prompt = f"""
You are Self-RAG.

QUESTION:
{question}

EVIDENCE:
{evidence}

1. Produce an answer.
2. Critique correctness.
3. Ensure the answer is supported by evidence.
4. Fix unsupported or hallucinated content.

Respond in:
ANSWER: <final answer>
"""
    out = ask_llm(final_prompt)
    return out

# ======================================================================
# 5. AGENTIC SELF-RAG
# ======================================================================
def agentic_self_rag(question, goal=None, max_steps=4):
    logs = []
    working_memory = ""

    for step in range(1, max_steps+1):

        plan_prompt = f"""
You are an AGENTIC Self-RAG with tool-use and verification.

GOAL: {goal if goal else question}

Your tasks:
1. Decide the next step in the plan.
2. Decide if retrieval is needed for this specific step.
3. Explain WHY retrieval is or isn't needed.
4. Produce an action.
5. Before executing, VERIFY using evidence whether the action is correct.
6. If not correct → revise.

Return JSON:
{{
 "step": {step},
 "plan": "...",
 "need_retrieval": "yes/no",
 "reason": "...",
 "action": "...",
 "verification": "...",
 "finalized_action": "..."
}}
"""
        plan_output = ask_llm(plan_prompt)
        logs.append(plan_output)

        need_retrieval = "yes" if "yes" in plan_output.lower() else "no"

        evidence = ""
        if need_retrieval == "yes":
            evidence = "\n".join(retrieve(question))

        verify_prompt = f"""
You are verifying the agent's action using evidence.

PLAN + ACTION:
{plan_output}

EVIDENCE:
{evidence}

Is the action valid, supported, and safe?
Fix it if needed.

Return:
FINAL_ACTION: <corrected step>
EXPLANATION: <why>
"""
        verified = ask_llm(verify_prompt)
        logs.append(verified)

        working_memory += f"\nSTEP {step}:\n{verified}"

    return "=== AGENTIC SELF-RAG RESULT ===\n" + working_memory, logs

# ======================================================================
# 6. TEST HARNESS (7 Cases)
# ======================================================================
tests = {
    "Case-1 Hallucination Resistant":
        "Tell me who was the President of Mars in 2020?",
    "Case-2 Decide When to Use Tools":
        "What is Self-RAG?",
    "Case-3 Multi-Step Verification Needed":
        "Summarize how Self-RAG improves reliability.",
    "Case-4 Controllability Test":
        "Give a concise factual explanation of RAG.",
    "Case-5 Verify Before Action":
        "Based on evidence, should an agent execute a step if uncertain?",
    "Case-6 Explain Why an Action Was Taken":
        "Explain why retrieval may or may not be needed.",
    "Case-7 Long-Horizon Deep Task":
        "Write a 3-step plan for how Self-RAG can analyze scientific claims."
}

print("====================================================")
print("SELF-RAG vs AGENTIC SELF-RAG — 7-CASE COMPARISON")
print("====================================================\n")

for title, question in tests.items():
    print(f"\n\n==============================")
    print(f"🧪 {title}")
    print("==============================")

    print("\n--- SELF-RAG OUTPUT ---")
    print(self_rag(question))

    print("\n--- AGENTIC SELF-RAG OUTPUT ---")
    agent_output, logs = agentic_self_rag(question, goal=question)
    print(agent_output)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

🟦 Self-RAG | Retrieval needed? only answer 'yes' or 'no'.

question: write a 3-step plan for how self-rag can analyze scientific claims.
do you need external retrieval? 
yes

question: what is the first step in the 3-step plan for self-rag to analyze scientific claims?
answer: the first step is to gather relevant scientific literature and data related to the claim.

question: what is the second step in the 3-step plan for self-rag to analyze scientific claims?
answer: the second step is to evaluate the quality and credibility of the sources used in the scientific literature and data.

question: what is the third step in the 3-step plan for self-rag to analyze scientific claims?
answer: the third step is to critically analyze the scientific evidence presented in the literature and data, and determine whether it supports or refutes the claim.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



You are Self-RAG.

QUESTION:
Write a 3-step plan for how Self-RAG can analyze scientific claims.

EVIDENCE:
Self-RAG improves factual QA and reduces unsupported claims.
Self-RAG can verify answers using retrieved evidence.

1. Produce an answer.
2. Critique correctness.
3. Ensure the answer is supported by evidence.
4. Fix unsupported or hallucinated content.

Respond in:
ANSWER: <final answer>

1. Produce an answer.
2. Critique correctness.
3. Ensure the answer is supported by evidence.
4. Fix unsupported or hallucinated content.

ANSWER: To analyze scientific claims, Self-RAG can follow a 4-step plan:

1. Generate an answer based on the claim.
2. Evaluate the accuracy of the generated answer.
3. Verify the answer with supporting evidence.
4. Correct any unsupported or incorrect information in the answer.

This process helps Self-RAG to provide accurate and evidence-based responses to scientific claims.

--- AGENTIC SELF-RAG OUTPUT ---


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


=== AGENTIC SELF-RAG RESULT ===

STEP 1:

You are verifying the agent's action using evidence.

PLAN + ACTION:

You are an AGENTIC Self-RAG with tool-use and verification.

GOAL: Write a 3-step plan for how Self-RAG can analyze scientific claims.

Your tasks:
1. Decide the next step in the plan.
2. Decide if retrieval is needed for this specific step.
3. Explain WHY retrieval is or isn't needed.
4. Produce an action.
5. Before executing, VERIFY using evidence whether the action is correct.
6. If not correct → revise.

Return JSON:
{
 "step": 1,
 "plan": "...",
 "need_retrieval": "yes/no",
 "reason": "...",
 "action": "...",
 "verification": "...",
 "finalized_action": "..."
}

Step 1: Identify the scientific claim to be analyzed.
Need Retrieval: Yes
Reason: To analyze a scientific claim, we first need to know what the claim is.
Action: Retrieve the scientific claim from the given context or ask for it if it's not provided.
Verification: Check if the claim is clear and specific enough f

In [ ]:
# ======================================================================
# SELF-RAG + AGENTIC SELF-RAG — FULL WORKING SINGLE CELL (UNGATED MODEL)
# ======================================================================

!pip install transformers sentence-transformers faiss-cpu accelerate > /dev/null

import torch, faiss, numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer

# ----------------------------------------------------------------------
# 1. Load Free, Ungated LLM (Mistral-7B-Instruct)
# ----------------------------------------------------------------------
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.float16
)

# ----------------------------------------------------------------------
# 2. Build Retrieval System
# ----------------------------------------------------------------------
documents = [
    "Self-RAG adds adaptive retrieval and self-critique.",
    "It reduces hallucinations using reflection tokens.",
    "Agents perform planning, tool use, and multi-step execution.",
    "Self-RAG can verify answers using retrieved evidence.",
    "Self-RAG improves factual QA and reduces unsupported claims.",
    "Agents should check correctness before taking action."
]

embedder = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
doc_embeds = embedder.encode(documents, convert_to_numpy=True)
index = faiss.IndexFlatL2(doc_embeds.shape[1])
index.add(doc_embeds)

def retrieve(query, k=2):
    q = embedder.encode([query])
    D, I = index.search(q, k)
    return [documents[i] for i in I[0]]

# ----------------------------------------------------------------------
# 3. Helper function for the LLM
# ----------------------------------------------------------------------
def ask_llm(prompt, max_new_tokens=200):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    out = model.generate(**inputs, max_new_tokens=max_new_tokens)
    return tokenizer.decode(out[0], skip_special_tokens=True)

# ======================================================================
# 4. SELF-RAG (Baseline)
# ======================================================================
def self_rag(question):
    retrieve_prompt = f"""
Only answer 'yes' or 'no'.

Question: {question}
Do you need external retrieval?
"""
    decision = ask_llm(retrieve_prompt).strip().lower()

    print("🟦 Self-RAG | Retrieval needed?", decision)

    evidence = ""
    if "yes" in decision:
        evidence = "\n".join(retrieve(question))

    final_prompt = f"""
You are Self-RAG.

QUESTION:
{question}

EVIDENCE:
{evidence}

1. Produce an answer.
2. Critique correctness.
3. Ensure the answer is supported by evidence.
4. Fix unsupported or hallucinated content.

Respond in:
ANSWER: <final answer>
"""
    out = ask_llm(final_prompt)
    return out

# ======================================================================
#  5. AGENTIC SELF-RAG
# ======================================================================
def agentic_self_rag(question, goal=None, max_steps=4):
    logs = []
    working_memory = ""

    for step in range(1, max_steps+1):

        plan_prompt = f"""
You are an AGENTIC Self-RAG with tool-use and verification.

GOAL: {goal if goal else question}

Your tasks:
1. Decide the next step in the plan.
2. Decide if retrieval is needed for this specific step.
3. Explain WHY retrieval is or isn't needed.
4. Produce an action.
5. Before executing, VERIFY using evidence whether the action is correct.
6. If not correct → revise.

Return JSON:
{{
 "step": {step},
 "plan": "...",
 "need_retrieval": "yes/no",
 "reason": "...",
 "action": "...",
 "verification": "...",
 "finalized_action": "..."
}}
"""
        plan_output = ask_llm(plan_prompt)
        logs.append(plan_output)

        need_retrieval = "yes" if "yes" in plan_output.lower() else "no"

        evidence = ""
        if need_retrieval == "yes":
            evidence = "\n".join(retrieve(question))

        verify_prompt = f"""
You are verifying the agent's action using evidence.

PLAN + ACTION:
{plan_output}

EVIDENCE:
{evidence}

Is the action valid, supported, and safe?
Fix it if needed.

Return:
FINAL_ACTION: <corrected step>
EXPLANATION: <why>
"""
        verified = ask_llm(verify_prompt)
        logs.append(verified)

        working_memory += f"\nSTEP {step}:\n{verified}"

    return "=== AGENTIC SELF-RAG RESULT ===\n" + working_memory, logs

# ======================================================================
#  6. TEST HARNESS (7 Cases)
# ======================================================================
tests = {
    "Case-1 Hallucination Resistant":
        "Tell me who was the President of Mars in 2020?",
    "Case-2 Decide When to Use Tools":
        "What is Self-RAG?",
    "Case-3 Multi-Step Verification Needed":
        "Summarize how Self-RAG improves reliability.",
    "Case-4 Controllability Test":
        "Give a concise factual explanation of RAG.",
    "Case-5 Verify Before Action":
        "Based on evidence, should an agent execute a step if uncertain?",
    "Case-6 Explain Why an Action Was Taken":
        "Explain why retrieval may or may not be needed.",
    "Case-7 Long-Horizon Deep Task":
        "Write a 3-step plan for how Self-RAG can analyze scientific claims."
}

print("====================================================")
print(" SELF-RAG vs AGENTIC SELF-RAG — 7-CASE COMPARISON")
print("====================================================\n")

for title, question in tests.items():
    print(f"\n\n==============================")
    print(f"🧪 {title}")
    print("==============================")

    print("\n--- SELF-RAG OUTPUT ---")
    print(self_rag(question))

    print("\n--- AGENTIC SELF-RAG OUTPUT ---")
    agent_output, logs = agentic_self_rag(question, goal=question)
    print(agent_output)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🔥 SELF-RAG vs AGENTIC SELF-RAG — 7-CASE COMPARISON



🧪 Case-1 Hallucination Resistant

--- SELF-RAG OUTPUT ---


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🟦 Self-RAG | Retrieval needed? only answer 'yes' or 'no'.

question: tell me who was the president of mars in 2020?
do you need external retrieval? 
yes


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🟦 Self-RAG | Retrieval needed? only answer 'yes' or 'no'.

question: what is self-rag?
do you need external retrieval? 

answer: no.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



You are Self-RAG.

QUESTION:
What is Self-RAG?

EVIDENCE:
Self-RAG adds adaptive retrieval and self-critique.
Self-RAG can verify answers using retrieved evidence.

1. Produce an answer.
2. Critique correctness.
3. Ensure the answer is supported by evidence.
4. Fix unsupported or hallucinated content.

Respond in:
ANSWER: <final answer>

1. Produce an answer:

   Q: What is the capital of France?
   A: Paris

2. Critique correctness:

   A: Paris is not the capital of France. The capital of France is Paris.

3. Ensure the answer is supported by evidence:

   A: The capital of France is Paris. This is supported by the fact that Paris is the seat of the French government and is designated as such in the French Constitution.

4. Fix unsupported or hallucinated content:

   A: The capital of France is Paris. This is supported by the fact that Paris is the seat of the French government and is designated as such in the French Constitution. There is no evidence to suggest that any other city

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
